# Optimal Whiskey Mixtures

<b>Goal.</b> Set up and solve an LP with `pulp`.

An importer for whiskey is faced with an unlimited market for his products. However, his monthly amount to purchase is limited by import constraints as follows:
- <i>Sir Roses</i>:  at most 2000 liters for 35 CHF each
- <i>Highland Wind</i>:  at most 2500 liters for 25 CHF each
- <i>Old Frenzy</i>:  at most 1200 liters for 20 CHF each

Based on this, he produces the mixtures A, B, and C that he sells for 34 CHF, 28 CHF, and 23 CHF per liter, respectively. The mixtures are composed by the following ratios:
- Mixture A: at least 60% <i>Sir Roses</i>, at most 20% <i>Old Frenzy</i>
- Mixture B: at least 15% <i>Sir Roses</i>, at most 60% <i>Old Frenzy</i>
- Mixture C: at most 50% <i>Old Frenzy</i>

Now the importer needs to know how he should compose the mixtures A, B, and C and how much should be produced of each as to maximize his profit.

<b>Your Task:</b> Advise the importer to maximize his profit! Determine the solution by implementing an LP with `pulp`.

<b>Step 1:</b> Set up the LP.

In [1]:
import pulp

# Set up the optimization program

mylp = pulp.LpProblem("Whiskey Mixtures", pulp.LpMaximize)

<b>Step 2:</b> Introduce suitable variables and describe them briefly.

We introduce the nine variables

$$x_{i,j} \geq 0 \text{ the number of liters of spirit $i$ used in mixture $j$ for $i\in \{SR,HW,OF\}$ and $j\in \{A,B,C\}$}$$

The imported amount of a spirit, say <i>Highland Wind</i>, will then be the sum $x_{HW,A} + x_{HW,B} + x_{HW,C}$ and the prodcued amount of a mixture, say mixture C, will be $x_{SR,C} + x_{HW,C} + x_{OF,C}$.

In [2]:
# Create the variables

SR = {i: pulp.LpVariable(f'SR{i}', lowBound=0) for i in ["A","B","C"]} #l of Sir Roses used for A, B, C
HW = {i: pulp.LpVariable(f'HW{i}', lowBound=0) for i in ["A","B","C"]} #l of Highland Wind used for A, B, C
OF = {i: pulp.LpVariable(f'OF{i}', lowBound=0) for i in ["A","B","C"]} #l of Old Frenzy used for A, B, C

<b>Step 3:</b> Write down and implement the objective function.

The objective function is composed of the sales minus the expenses:

\begin{array}{rcrcrc}
\max	    & + 34&(x_{SR,A} &+& x_{HW,A} &+& x_{OF,A}) \\
            & + 28&(x_{SR,B} &+& x_{HW,B} &+& x_{OF,B}) \\
            & + 23&(x_{SR,C} &+& x_{HW,C} &+& x_{OF,C}) \\
            & - 35&(x_{SR,A} &+& x_{SR,B} &+& x_{SR,C}) \\
            & - 25&(x_{HW,A} &+& x_{HW,B} &+& x_{HW,C}) \\
            & - 20&(x_{OF,A} &+& x_{OF,B} &+& x_{OF,C})
\end{array}

In [3]:
# Add the objective function

mylp += 34*(SR["A"] + HW["A"] + OF["A"]) + 28*(SR["B"] + HW["B"] + OF["B"]) + 23*(SR["C"] + HW["C"] + OF["C"]) \
      - 35*(SR["A"] + SR["B"] + SR["C"]) - 25*(HW["A"] + HW["B"] + HW["C"]) - 20*(OF["A"] + OF["B"] + OF["C"])

<b>Step 4:</b> Write down and implement the constraints concerning the import restrictions and the composition requirements.

The import restrictions are

\begin{array}{rcrcrc}
            & x_{SR,A} &+& x_{SR,B} &+& x_{SR,C} &\leq &2000 \\
            & x_{HW,A} &+& x_{HW,B} &+& x_{HW,C} &\leq &2500 \\
            & x_{OF,A} &+& x_{OF,B} &+& x_{OF,C} &\leq &1200
\end{array}

In [4]:
# Add the constraints concerning the import restrictions

mylp += SR["A"] + SR["B"] + SR["C"] <= 2000
mylp += HW["A"] + HW["B"] + HW["C"] <= 2500
mylp += OF["A"] + OF["B"] + OF["C"] <= 1200

The composition requirements are

\begin{array}{rcrcrc}
           x_{SR,A} &\geq& 0.6&(x_{SR,A} &+& x_{HW,A} &+& x_{OF,A}) \\
           x_{OF,A} &\leq& 0.2&(x_{SR,A} &+& x_{HW,A} &+& x_{OF,A}) \\
           x_{SR,B} &\geq& 0.15&(x_{SR,B} &+& x_{HW,B} &+& x_{OF,B}) \\
           x_{OF,B} &\leq& 0.6&(x_{SR,B} &+& x_{HW,B} &+& x_{OF,B}) \\
           x_{OF,C} &\leq& 0.5&(x_{SR,C} &+& x_{HW,C} &+& x_{OF,C})
\end{array}

In [5]:
# Add the constraints concerning the composition requirements

mylp += SR["A"] >= 0.6*(SR["A"] + HW["A"] + OF["A"])
mylp += OF["A"] <= 0.2*(SR["A"] + HW["A"] + OF["A"])
mylp += SR["B"] >= 0.15*(SR["B"] + HW["B"] + OF["B"])
mylp += OF["B"] <= 0.6*(SR["B"] + HW["B"] + OF["B"])
mylp += OF["C"] <= 0.5*(SR["C"] + HW["C"] + OF["C"])

<b>Step 5:</b> Solve the LP and give the maximal profit. Find out how much the importer should produce of each mixture and how they are composed.

In [6]:
# Solve the LP and give the optimal objective value (i.e. the maximal profit that can be achieved)

mylp.solve()
mylp.objective.value()

18366.66679

In [7]:
# Print how much the importer should produce of each mixture and how they are composed

import numpy as np
for i in ["A","B","C"]:
    m = [SR[i].value(), HW[i].value(), OF[i].value()] #mixture
    print("Produce " + str(round(sum(m))) + " l of mixture " + i + \
          " according to the composition: " + str(np.round(m)))


Produce 2544 l of mixture A according to the composition: [1527. 1018.    0.]
Produce 3156 l of mixture B according to the composition: [ 473. 1482. 1200.]
Produce 0 l of mixture C according to the composition: [0. 0. 0.]
